# Project Firefox MK I

In [53]:
from pprint import pprint
#calling library
from kinase_declarative import *
from sqlalchemy import create_engine, or_
from sqlalchemy.orm import sessionmaker

In [54]:
#create engine and bine the engine
engine = create_engine("sqlite:///kinase_database.db")
Base.metadata.bind = engine

In [55]:
#create a session object
session = sessionmaker(bind=engine)
s = session()

# Table of contents

1. [Get Uniprot gene_name and alias related to a kinase of query](#Get-Uniprot-gene_name-and-alias-related-to-a-kinase-of-query)
2. [Get meta data for a gene name](#Get-meta-data-for-a-gene-name)
3. [Get subcellular location for a gene](#Get-subcellular-location-for-a-gene)
4. [Get inhibitor for a gene](#Get-inhibitor-for-a-gene)
5. [Get the phosphosites data through the substrates](#Get-the-phosphosites-data-through-the-substrates)
6. [Get the phosphosites through the kinase](#Get-the-phosphosites-through-the-kinase)
7. [Get substrates from the kinase](#Get-substrates-from-the-kinase)
8. [Get the phosphosites meta and the substrates meta data through a kinase gene](#Get-the-phosphosites-meta-and-the-substrates-meta-data-through-a-kinase-gene)
9. [Work in progress, do not disturb](#Work-in-progress,-do-not-disturb)

Click on any of the link to go to the section of the codes that contains the function.

## Get Uniprot gene_name and alias related to a kinase of query

In [75]:
#search for things related to MAPK1
kinase = "MAPK1"
kinase_query = s.query(KinaseGeneName).filter(KinaseGeneName.gene_alias==kinase).one()
kinase_query.to_dict()

{'gene_name': 'MAPK1', 'gene_alias': 'MAPK1'}

In [5]:
def get_uniprot_gene_name(kinase):
    """
    Returns the preferred uniprot gene name.
    Returns a dictionary.
    """
    kinase_match = s.query(KinaseGeneName).filter(KinaseGeneName.gene_alias==kinase).one()
    return kinase_match.to_dict()
    

In [6]:
#testing the functions
kinase = "MAPK1"
print(get_uniprot_gene_name(kinase))

{'gene_name': 'MAPK1', 'gene_alias': 'MAPK1'}


In [7]:
kinase="PRKM1"
print(get_uniprot_gene_name(kinase))

{'gene_name': 'MAPK1', 'gene_alias': 'PRKM1'}


In [8]:
#test error
kinase = "error"
print(get_uniprot_gene_name(kinase))
#this will raise error when run

NoResultFound: No row was found for one()

## Get meta data for a gene name

In [9]:
#get the meta data for a kinase whose name is the preferred name of uniprot
kinase = 'MAPK1'
kinase_query = s.query(KinaseGeneMeta).filter(KinaseGeneMeta.gene_name==kinase).one()
kinase_query.to_dict()

{'protein_name': 'Mitogen-activated protein kinase 1',
 'uniprot_number': 'P28482',
 'uniprot_entry': 'MK01_HUMAN',
 'gene_name': 'MAPK1',
 'kinase_family': 'CMGC Ser/Thr protein kinase family'}

In [10]:
#getting the meta data of a kinase who name is an alias
kinase = "PRKM1"
kinase_query = s.query(KinaseGeneMeta).join(KinaseGeneName).filter(KinaseGeneName.gene_alias==kinase).one()
kinase_query.to_dict()

{'protein_name': 'Mitogen-activated protein kinase 1',
 'uniprot_number': 'P28482',
 'uniprot_entry': 'MK01_HUMAN',
 'gene_name': 'MAPK1',
 'kinase_family': 'CMGC Ser/Thr protein kinase family'}

In [11]:
#getting the meta data of the kinase gene regardless of whether the name is the prefered gene name in uniprot or not
def get_meta_data(kinase):
    """
    Return the metadata of a gene based on its kinase gene name.
    It can be the prefered gene name or the alias/ synonyms.
    Returns an error if no gene was found.
    Returns a dictionary
    """
    kinase_query = s.query(KinaseGeneMeta).join(KinaseGeneName).filter(KinaseGeneName.gene_alias==kinase).one()
    return kinase_query.to_dict()

In [12]:
kinase = "KIAA0641"
print(get_meta_data(kinase))

{'protein_name': 'Serine/threonine-protein kinase LMTK1', 'uniprot_number': 'Q6ZMQ8', 'uniprot_entry': 'LMTK1_HUMAN', 'gene_name': 'AATK', 'kinase_family': 'Tyr protein kinase family'}


## Get subcellular location for a gene

In [59]:
kinase = "ALPK1"
kinase_query = s.query(KinaseSubcellularLocation).join(KinaseGeneName).filter(KinaseGeneName.gene_alias==kinase).all()
kinase_query[-1].to_dict()
for row in kinase_query:
    print(row.subcellular_location)

Cytosol
Cytoplasm


In [71]:
def get_subcellular_location(kinase):
    """
    Returns a list of subcellular location as an object for a kinase.
    """
    results = []
    kinase_query = s.query(KinaseSubcellularLocation).join(KinaseGeneName).filter(KinaseGeneName.gene_alias==kinase).all()
    for row in kinase_query:
        results.append(row.to_dict())
    return results
kinase = "ALPK1"
print(get_subcellular_location(kinase))

[{'subcellular_location_id': 129, 'gene_name': 'ALPK1', 'subcellular_location': 'Cytosol'}, {'subcellular_location_id': 130, 'gene_name': 'ALPK1', 'subcellular_location': 'Cytoplasm'}]


## Get inhibitor for a gene

In [67]:
#print out each object as a dictionary
kinase = "SGK1"
kinase_query = s.query(Inhibitor).join(KinaseGeneName).filter(KinaseGeneName.gene_alias==kinase).all()
for row in kinase_query:
    print(row.to_dict())

{'inhibitor_id': 1, 'inhibitor': 'GSK650394A', 'antagonizes_gene': 'SGK1', 'molecular_weight': 382.45, 'images_url': 'http://www.kinase-screen.mrc.ac.uk/system/files/compounds/jpg/gsk-50394_5.jpg', 'empirical_formula': 'C25H22N2O2'}
{'inhibitor_id': 2, 'inhibitor': 'SGK-Sanofi-14i', 'antagonizes_gene': 'SGK1', 'molecular_weight': 430.87, 'images_url': 'http://www.kinase-screen.mrc.ac.uk/system/files/compounds/jpg/SGK1-Sanofi-14i.jpg', 'empirical_formula': 'C18H15ClN6O3S'}
{'inhibitor_id': 3, 'inhibitor': 'SGK1-Sanofi-14g', 'antagonizes_gene': 'SGK1', 'molecular_weight': 435.29, 'images_url': 'http://www.kinase-screen.mrc.ac.uk/system/files/compounds/jpg/SGK1-Sanofi--14g.jpg', 'empirical_formula': 'C17H12Cl2N6O2S'}
{'inhibitor_id': 4, 'inhibitor': 'SGK1-Sanofi-14h', 'antagonizes_gene': 'SGK1', 'molecular_weight': 435.29, 'images_url': 'http://www.kinase-screen.mrc.ac.uk/system/files/compounds/jpg/SGK1-Sanofi-14h_0.jpg', 'empirical_formula': 'C17H12Cl2N6O2S'}
{'inhibitor_id': 5, 'inhibit

In [68]:
#print out each object's inhibitor
kinase = "SGK1"
kinase_query = s.query(Inhibitor).join(KinaseGeneName).filter(KinaseGeneName.gene_alias==kinase).all()
for row in kinase_query:
    print(row.inhibitor)

GSK650394A
SGK-Sanofi-14i
SGK1-Sanofi-14g
SGK1-Sanofi-14h
SGK1-Sanofi-14n


In [62]:
def get_inhibitor(kinase):
    """
    Return a list of dictionaries.
    """
    results = []
    kinase_query = s.query(Inhibitor).join(KinaseGeneName).filter(KinaseGeneName.gene_alias==kinase).all()
    for row in kinase_query:
        results.append(row.to_dict())
    return results
kinase = "SGK1"
get_inhibitor(kinase)

[{'inhibitor_id': 1,
  'inhibitor': 'GSK650394A',
  'antagonizes_gene': 'SGK1',
  'molecular_weight': 382.45,
  'images_url': 'http://www.kinase-screen.mrc.ac.uk/system/files/compounds/jpg/gsk-50394_5.jpg',
  'empirical_formula': 'C25H22N2O2'},
 {'inhibitor_id': 2,
  'inhibitor': 'SGK-Sanofi-14i',
  'antagonizes_gene': 'SGK1',
  'molecular_weight': 430.87,
  'images_url': 'http://www.kinase-screen.mrc.ac.uk/system/files/compounds/jpg/SGK1-Sanofi-14i.jpg',
  'empirical_formula': 'C18H15ClN6O3S'},
 {'inhibitor_id': 3,
  'inhibitor': 'SGK1-Sanofi-14g',
  'antagonizes_gene': 'SGK1',
  'molecular_weight': 435.29,
  'images_url': 'http://www.kinase-screen.mrc.ac.uk/system/files/compounds/jpg/SGK1-Sanofi--14g.jpg',
  'empirical_formula': 'C17H12Cl2N6O2S'},
 {'inhibitor_id': 4,
  'inhibitor': 'SGK1-Sanofi-14h',
  'antagonizes_gene': 'SGK1',
  'molecular_weight': 435.29,
  'images_url': 'http://www.kinase-screen.mrc.ac.uk/system/files/compounds/jpg/SGK1-Sanofi-14h_0.jpg',
  'empirical_formula':

In [65]:
pprint(get_inhibitor("PDPK1"))

[{'antagonizes_gene': 'PDPK1',
  'empirical_formula': 'C23H31BrN8O3',
  'images_url': 'http://www.kinase-screen.mrc.ac.uk/system/files/compounds/jpg/bx-320_4.jpg',
  'inhibitor': 'BX-320',
  'inhibitor_id': 9,
  'molecular_weight': 547.45},
 {'antagonizes_gene': 'PDPK1',
  'empirical_formula': 'C15H14N4O2',
  'images_url': 'http://www.kinase-screen.mrc.ac.uk/system/files/compounds/jpg/bx-517_4.jpg',
  'inhibitor': 'BX-517',
  'inhibitor_id': 10,
  'molecular_weight': 282.3},
 {'antagonizes_gene': 'PDPK1',
  'empirical_formula': 'C19H15N5O2',
  'images_url': 'http://www.kinase-screen.mrc.ac.uk/system/files/compounds/jpg/bx-517-analogue_4.jpg',
  'inhibitor': 'BX-517 Analog (compound 7b)',
  'inhibitor_id': 11,
  'molecular_weight': 345.35},
 {'antagonizes_gene': 'PDPK1',
  'empirical_formula': 'C23H26IN7O2S',
  'images_url': 'http://www.kinase-screen.mrc.ac.uk/system/files/compounds/jpg/bx-795_4.jpg',
  'inhibitor': 'BX-795',
  'inhibitor_id': 12,
  'molecular_weight': 591.47},
 {'antag

## Get the kinase gene through an inhibitor

In [79]:
#get the kinases through an inhibitor
inhibitor_name = "PD 184352 (CI-1040)"
inhibitor_query = s.query(Inhibitor).join(KinaseGeneName).filter(Inhibitor.inhibitor==inhibitor_name).\
filter(KinaseGeneName.gene_alias==Inhibitor.antagonizes_gene).all()
for row in inhibitor_query:
    pprint(row.kinases.gene_name)

'YES1'
'MAPK3'
'MAP2K1'


In [80]:
#create a function that gets the kinases for an inhibitor
def get_kinase_from_inhibitor(inhibitor_name):
    """
    Returns a list of kinases that are inhibited by the inhibitor.
    """
    results = []
    inhibitor = s.query(Inhibitor).join(KinaseGeneName).filter(Inhibitor.inhibitor==inhibitor_name).\
    filter(KinaseGeneName.gene_alias==Inhibitor.antagonizes_gene).all()
    for row in inhibitor:
        results.append(row.kinases.gene_name)
    return results

In [83]:
inhibitor = "GSK650394A"
print(get_kinase_from_inhibitor(inhibitor))
pprint(get_inhibitor("SGK1"))

['SGK1']
[{'antagonizes_gene': 'SGK1',
  'empirical_formula': 'C25H22N2O2',
  'images_url': 'http://www.kinase-screen.mrc.ac.uk/system/files/compounds/jpg/gsk-50394_5.jpg',
  'inhibitor': 'GSK650394A',
  'inhibitor_id': 1,
  'molecular_weight': 382.45},
 {'antagonizes_gene': 'SGK1',
  'empirical_formula': 'C18H15ClN6O3S',
  'images_url': 'http://www.kinase-screen.mrc.ac.uk/system/files/compounds/jpg/SGK1-Sanofi-14i.jpg',
  'inhibitor': 'SGK-Sanofi-14i',
  'inhibitor_id': 2,
  'molecular_weight': 430.87},
 {'antagonizes_gene': 'SGK1',
  'empirical_formula': 'C17H12Cl2N6O2S',
  'images_url': 'http://www.kinase-screen.mrc.ac.uk/system/files/compounds/jpg/SGK1-Sanofi--14g.jpg',
  'inhibitor': 'SGK1-Sanofi-14g',
  'inhibitor_id': 3,
  'molecular_weight': 435.29},
 {'antagonizes_gene': 'SGK1',
  'empirical_formula': 'C17H12Cl2N6O2S',
  'images_url': 'http://www.kinase-screen.mrc.ac.uk/system/files/compounds/jpg/SGK1-Sanofi-14h_0.jpg',
  'inhibitor': 'SGK1-Sanofi-14h',
  'inhibitor_id': 4,
  

## Get the phosphosites data through the substrates

In [97]:
substrate_gene = "ABL1"
phosphosite_query = s.query(PhosphositeMeta)\
.filter(PhosphositeMeta.substrate_meta_id==SubstrateMeta.substrate_id)\
.filter(SubstrateMeta.substrate_gene_name==substrate_gene).all()
for row in phosphosite_query:
    print(row.to_dict())

{'phosphosite_meta_id': 2877, 'substrate_meta_id': 1222, 'phosphosite': 'Y115', 'chromosome': 9, 'karyotype_band': 'q34.12', 'strand': 1, 'start_position': 130854890, 'end_position': 130854892, 'neighbouring_sequences': 'QGWVPsNyItPVNSL'}
{'phosphosite_meta_id': 2878, 'substrate_meta_id': 1222, 'phosphosite': 'Y70', 'chromosome': 9, 'karyotype_band': 'q34.12', 'strand': 1, 'start_position': 130854192, 'end_position': 130854194, 'neighbouring_sequences': 'PNLFVALyDFVAsGD'}
{'phosphosite_meta_id': 2879, 'substrate_meta_id': 1222, 'phosphosite': 'Y128', 'chromosome': 9, 'karyotype_band': 'q34.12', 'strand': 1, 'start_position': 130854929, 'end_position': 130854931, 'neighbouring_sequences': 'SLEKHSWyHGPVSRN'}
{'phosphosite_meta_id': 2880, 'substrate_meta_id': 1222, 'phosphosite': 'Y185', 'chromosome': 9, 'karyotype_band': 'q34.12', 'strand': 1, 'start_position': 130862766, 'end_position': 130862768, 'neighbouring_sequences': 'TAsDGKLyVsSESRF'}
{'phosphosite_meta_id': 2881, 'substrate_meta

The results above show all the phosphosites on the substrate with the gene name, ABL1.

In [100]:
#create a function to return the phosphosites of a substrate as a list
def get_phosphosites_from_substrates(substrate_gene):
    """
    Returns a list of dictionaries containing metadata of phosphosites on a substrate.
    """
    results = []
    phosphosite_query = s.query(PhosphositeMeta).filter(PhosphositeMeta.substrate_meta_id==SubstrateMeta.substrate_id)\
    .filter(SubstrateMeta.substrate_gene_name==substrate_gene).all()
    for row in phosphosite_query:
        results.append(row.to_dict())
    return results
substrate_gene = "MAPK1"
print(get_phosphosites_from_substrates(substrate_gene))

[{'phosphosite_meta_id': 1873, 'substrate_meta_id': 265, 'phosphosite': 'S29', 'chromosome': 22, 'karyotype_band': 'q11.22', 'strand': -1, 'start_position': 21867356, 'end_position': 21867354, 'neighbouring_sequences': 'GPRyTNLsYIGEGAy'}, {'phosphosite_meta_id': 1874, 'substrate_meta_id': 265, 'phosphosite': 'S248', 'chromosome': 22, 'karyotype_band': 'q11.22', 'strand': -1, 'start_position': 21788371, 'end_position': 21788369, 'neighbouring_sequences': 'LGILGsPsQEDLNCI'}, {'phosphosite_meta_id': 1875, 'substrate_meta_id': 265, 'phosphosite': 'S246', 'chromosome': 22, 'karyotype_band': 'q11.22', 'strand': -1, 'start_position': 21788377, 'end_position': 21788375, 'neighbouring_sequences': 'HILGILGsPsQEDLN'}, {'phosphosite_meta_id': 1876, 'substrate_meta_id': 265, 'phosphosite': 'T185', 'chromosome': 22, 'karyotype_band': 'q11.22', 'strand': -1, 'start_position': 21799068, 'end_position': 21799066, 'neighbouring_sequences': 'HDHtGFLtEyVAtRW'}]


In [ ]:
#to return a list of kinase connected to a substrates

In [ ]:
#to return a list of kinase connected to a substrate on a phosphosites based on a list of 
#substrates and its respective phosphosites

## Get the phosphosites through the kinase

In [ ]:
#get the phosphosites through the kinase
kinase = "MAPK1"
phosphosite_query = s.query(PhosphositeMeta).join(KinasePhosphositeRelations)\
.filter((KinasePhosphositeRelations.kinase_gene_id==kinase)&(PhosphositeMeta.phosphosite_meta_id==KinasePhosphositeRelations.phosphosite_id)).all()
for row in phosphosite_query:
    print(row.to_dict())

In [ ]:
#create a function to get the phosphosites through kinase
def get_phosphosites_from_kinase(kinase_gene):
    """
    Return the list of dictionaries containing metadata on phosphosites 
    that interact with a kinase.
    """
    results = []
    phosphosite_query = s.query(PhosphositeMeta).join(KinasePhosphositeRelations)\
    .filter((KinasePhosphositeRelations.kinase_gene_id==kinase)&(PhosphositeMeta.phosphosite_meta_id==KinasePhosphositeRelations.phosphosite_id)).all()
    for row in phosphosite_query:
        results.append(row.to_dict())
    return results
kinase = "MAPK1"
print(get_phosphosites_from_kinase(kinase))

## Get substrates from the kinase

In [ ]:
#get substrates
kinase = "MAPK1"
substrate_query = s.query(SubstrateMeta)\
.filter((KinasePhosphositeRelations.kinase_gene_id==kinase) & (KinasePhosphositeRelations.phosphosite_id==PhosphositeMeta.phosphosite_meta_id))\
.filter(PhosphositeMeta.substrate_meta_id==SubstrateMeta.substrate_id).all()
for row in substrate_query:
    print(row.to_dict())

The result aboove shows a list of dictionaries containing the metadata of the substrates that are/ were phosphorylated by the kinase, MAPK1.

NOTE: The function below only retrieves the meta data of the substrates phosphorylated by the kinase without retrieving the phosphosites for each kinase

In [107]:
#get the substrates from the kinase
def get_substrates_from_kinase_gene(kinase_gene):
    """
    Returns a list of dictionaries containing metadata of the substrates 
    phosphorylated by a kinase.
    """
    results = []
    substrate_query = s.query(SubstrateMeta).filter((KinasePhosphositeRelations.kinase_gene_id==kinase) & (KinasePhosphositeRelations.phosphosite_id==PhosphositeMeta.phosphosite_meta_id))\
    .filter(PhosphositeMeta.substrate_meta_id==SubstrateMeta.substrate_id).all()
    for row in substrate_query:
        results.append(row.to_dict())
    return results
print(get_substrates_from_kinase_gene(kinase))

[{'substrate_id': 1827, 'substrate_name': 'p90RSK', 'substrate_gene_name': 'RPS6KA1', 'substrate_uniprot_entry': 'KS6A1_HUMAN', 'substrate_uniprot_number': 'Q15418'}, {'substrate_id': 1842, 'substrate_name': 'p67phox', 'substrate_gene_name': 'NCF2', 'substrate_uniprot_entry': 'NCF2_HUMAN', 'substrate_uniprot_number': 'P19878'}, {'substrate_id': 1852, 'substrate_name': 'cPLA2', 'substrate_gene_name': 'PLA2G4A', 'substrate_uniprot_entry': 'PA24A_HUMAN', 'substrate_uniprot_number': 'P47712'}, {'substrate_id': 1828, 'substrate_name': 'PARVA', 'substrate_gene_name': 'PARVA', 'substrate_uniprot_entry': 'PARVA_HUMAN', 'substrate_uniprot_number': 'Q9NVD7'}, {'substrate_id': 1831, 'substrate_name': 'DUSP16', 'substrate_gene_name': 'DUSP16', 'substrate_uniprot_entry': 'DUS16_HUMAN', 'substrate_uniprot_number': 'Q9BY84'}, {'substrate_id': 1841, 'substrate_name': 'Fos', 'substrate_gene_name': 'FOS', 'substrate_uniprot_entry': 'FOS_HUMAN', 'substrate_uniprot_number': 'P01100'}, {'substrate_id': 183

In [22]:
#alternative
#work in progress
substrates_dict = {}
kinase_gene = "MAPK1"
kinase = s.query(KinaseGeneName).filter(KinaseGeneName.gene_alias==kinase_gene).one()
for phosphosite in kinase.phosphosites:
    substrates_dict[phosphosite.substrate] = phosphosite.substrate.to_dict()
substrates = list(substrates_dict.values())
pprint(substrates)

[{'phosphosites': [{'chromosome': 1,
                    'end_position': 26571577,
                    'karyotype_band': 'p36.11',
                    'neighbouring_sequences': 'AENGLLMtPCytANF',
                    'phosphosite': 'T573',
                    'phosphosite_meta_id': 92,
                    'start_position': 26571575,
                    'strand': 1,
                    'substrate_meta_id': 1827},
                   {'chromosome': 1,
                    'end_position': 26558862,
                    'karyotype_band': 'p36.11',
                    'neighbouring_sequences': 'HQLFRGFsFVAtGLM',
                    'phosphosite': 'S380',
                    'phosphosite_meta_id': 93,
                    'start_position': 26558860,
                    'strand': 1,
                    'substrate_meta_id': 1827}],
  'substrate_gene_name': 'RPS6KA1',
  'substrate_id': 1827,
  'substrate_name': 'p90RSK',
  'substrate_uniprot_entry': 'KS6A1_HUMAN',
  'substrate_uniprot_number': 'Q154

## Get the phosphosites meta and the substrates meta data through a kinase gene

In [21]:
kinase_gene = "MAPK1"
results = []
kinase = s.query(KinaseGeneName).filter(KinaseGeneName.gene_alias==kinase_gene).one()
dir(kinase_query) #this shows the attributes of the kinase
for phosphosite in kinase.phosphosites:
    results.append(phosphosite.substrate.to_dict())
results

[{'substrate_id': 1827,
  'substrate_name': 'p90RSK',
  'substrate_gene_name': 'RPS6KA1',
  'substrate_uniprot_entry': 'KS6A1_HUMAN',
  'substrate_uniprot_number': 'Q15418',
  'phosphosites': [{'phosphosite_meta_id': 92,
    'substrate_meta_id': 1827,
    'phosphosite': 'T573',
    'chromosome': 1,
    'karyotype_band': 'p36.11',
    'strand': 1,
    'start_position': 26571575,
    'end_position': 26571577,
    'neighbouring_sequences': 'AENGLLMtPCytANF'},
   {'phosphosite_meta_id': 93,
    'substrate_meta_id': 1827,
    'phosphosite': 'S380',
    'chromosome': 1,
    'karyotype_band': 'p36.11',
    'strand': 1,
    'start_position': 26558860,
    'end_position': 26558862,
    'neighbouring_sequences': 'HQLFRGFsFVAtGLM'}]},
 {'substrate_id': 1827,
  'substrate_name': 'p90RSK',
  'substrate_gene_name': 'RPS6KA1',
  'substrate_uniprot_entry': 'KS6A1_HUMAN',
  'substrate_uniprot_number': 'Q15418',
  'phosphosites': [{'phosphosite_meta_id': 92,
    'substrate_meta_id': 1827,
    'phosphosi

In [23]:
def get_substrate_and_phosphosite_through_kinase(kinase_gene):
    """
    Returns a list of dictionaries (outer).
    Each dictionary contains the key: substrate_id, substrate_name, substrate_gene_name, substrate_uniprot_entry, 
    substrate_uniprot_number, phosphosites.
    Phosphosites is a list of dictionary (inner). Each dictionary contains the meta data of the phosphosite on that substrate 
    within the same outer dictionary.
    If there are no substrates or phosphosites, it'll return an empty list.
    """
    results = []
    kinase = s.query(KinaseGeneName).filter(KinaseGeneName.gene_alias==kinase_gene).one()
    for phosphosite in kinase.phosphosites:
        results.append(phosphosite.substrate.to_dict())
    return results

In [26]:
kinase_gene = "GRK1"
pprint(get_substrate_and_phosphosite_through_kinase(kinase_gene))

[]


There are no substrates or phosphosites data for GRK1.

# Work in progress, do not disturb

In [33]:
kinase_search = "AKT3_HUMAN"
kinase_query = s.query(KinaseGeneName).filter(or_(KinaseGeneName.gene_alias==kinase_search, KinaseGeneMeta.uniprot_entry==kinase_search,\
                                   KinaseGeneMeta.uniprot_number==kinase_search)).all()
for row in kinase_query:
    pprint(row.to_dict())
#not quite

{'gene_alias': 'AKT1', 'gene_name': 'AKT1'}
{'gene_alias': 'PKB', 'gene_name': 'AKT1'}
{'gene_alias': 'RAC', 'gene_name': 'AKT1'}
{'gene_alias': 'AKT2', 'gene_name': 'AKT2'}
{'gene_alias': 'AKT3', 'gene_name': 'AKT3'}
{'gene_alias': 'PKBG', 'gene_name': 'AKT3'}
{'gene_alias': 'CDC42BPA', 'gene_name': 'CDC42BPA'}
{'gene_alias': 'KIAA0451', 'gene_name': 'CDC42BPA'}
{'gene_alias': 'CDC42BPB', 'gene_name': 'CDC42BPB'}
{'gene_alias': 'KIAA1124', 'gene_name': 'CDC42BPB'}
{'gene_alias': 'CDC42BPG', 'gene_name': 'CDC42BPG'}
{'gene_alias': 'DMPK2', 'gene_name': 'CDC42BPG'}
{'gene_alias': 'CIT', 'gene_name': 'CIT'}
{'gene_alias': 'CRIK', 'gene_name': 'CIT'}
{'gene_alias': 'KIAA0949', 'gene_name': 'CIT'}
{'gene_alias': 'STK21', 'gene_name': 'CIT'}
{'gene_alias': 'DMPK', 'gene_name': 'DMPK'}
{'gene_alias': 'DM1PK', 'gene_name': 'DMPK'}
{'gene_alias': 'MDPK', 'gene_name': 'DMPK'}
{'gene_alias': 'GRK1', 'gene_name': 'GRK1'}
{'gene_alias': 'RHOK', 'gene_name': 'GRK1'}
{'gene_alias': 'GRK2', 'gene_nam

In [56]:
kinase_search = "AKT3_HUMAN"
kinase_query = s.query(KinaseGeneMeta).filter(KinaseGeneMeta.uniprot_entry==kinase_search).one()
kinase_query.to_dict()

{'protein_name': 'RAC-gamma serine/threonine-protein kinase',
 'uniprot_number': 'Q9Y243',
 'uniprot_entry': 'AKT3_HUMAN',
 'gene_name': 'AKT3',
 'kinase_family': 'AGC Ser/Thr protein kinase family'}

In [57]:
kinase_search = "Q9Y243"
kinase_query = s.query(KinaseGeneMeta).filter(or_(KinaseGeneName.gene_alias==kinase_search, KinaseGeneMeta.uniprot_entry==kinase_search,\
                                   KinaseGeneMeta.uniprot_number==kinase_search)).all()
for row in kinase_query:
    pprint(row.to_dict())

{'gene_name': 'AKT3',
 'kinase_family': 'AGC Ser/Thr protein kinase family',
 'protein_name': 'RAC-gamma serine/threonine-protein kinase',
 'uniprot_entry': 'AKT3_HUMAN',
 'uniprot_number': 'Q9Y243'}


The above codes show that the or_ could work if you're using in the right table.

In [58]:
#to search using gene_alias
kinase_search = "KIAA1124"
kinase_query = s.query(KinaseGeneMeta).join(KinaseGeneName).filter(or_(KinaseGeneName.gene_alias==kinase_search, KinaseGeneMeta.uniprot_entry==kinase_search,\
                                   KinaseGeneMeta.uniprot_number==kinase_search, KinaseGeneMeta.protein_name==kinase_search)).all()
for row in kinase_query:
    pprint(row.to_dict())

{'gene_name': 'CDC42BPB',
 'kinase_family': 'AGC Ser/Thr protein kinase family',
 'protein_name': 'Serine/threonine-protein kinase MRCK beta',
 'uniprot_entry': 'MRCKB_HUMAN',
 'uniprot_number': 'Q9Y5S2'}


In [59]:
#to search using protein_name
kinase_search = "Serine/threonine-protein kinase MRCK beta"
kinase_query = s.query(KinaseGeneMeta).join(KinaseGeneName).filter(or_(KinaseGeneName.gene_alias==kinase_search, KinaseGeneMeta.uniprot_entry==kinase_search,\
                                   KinaseGeneMeta.uniprot_number==kinase_search, KinaseGeneMeta.protein_name==kinase_search)).all()
for row in kinase_query:
    pprint(row.to_dict())

{'gene_name': 'CDC42BPB',
 'kinase_family': 'AGC Ser/Thr protein kinase family',
 'protein_name': 'Serine/threonine-protein kinase MRCK beta',
 'uniprot_entry': 'MRCKB_HUMAN',
 'uniprot_number': 'Q9Y5S2'}


In [60]:
#search using uniprot entry
kinase_search = "MRCKB_HUMAN"
kinase_query = s.query(KinaseGeneMeta).join(KinaseGeneName).filter(or_(KinaseGeneName.gene_alias==kinase_search, KinaseGeneMeta.uniprot_entry==kinase_search,\
                                   KinaseGeneMeta.uniprot_number==kinase_search, KinaseGeneMeta.protein_name==kinase_search)).all()
for row in kinase_query:
    pprint(row.to_dict())

{'gene_name': 'CDC42BPB',
 'kinase_family': 'AGC Ser/Thr protein kinase family',
 'protein_name': 'Serine/threonine-protein kinase MRCK beta',
 'uniprot_entry': 'MRCKB_HUMAN',
 'uniprot_number': 'Q9Y5S2'}


# Functions demanded